<a href="https://colab.research.google.com/github/cagBRT/promptEngineering/blob/main/demo_distilgpt2_magicprompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> demo: distilgpt2-magicprompt

- [link to model card](https://huggingface.co/pszemraj/distilgpt2-magicprompt-SD)
- by [Peter](https://github.com/pszemraj)

---

In [1]:
pip install -U -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.2 MB/s eta 0:00:00


In [2]:
import re
from transformers import pipeline

model_id = "pszemraj/distilgpt2-magicprompt-SD"

generator = pipeline(
    "text-generation",
    model=model_id,
    device=-1,  # cpu inference takes 2 seconds
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [3]:

#@title define `augment_text` as helper

def clean_text(text):
    text = re.sub(r"\s+", " ", text)  # remove extra whitespace
    text = re.sub(r"\s+([?.!,])", r"\1", text)  # remove whitespace before punctuation
    # remove duplicate punctuation
    text = re.sub(r"([?.!,])\1+", r"\1", text)

    return text

def augment_text(
    prompt: str,
    pipeline,
    max_new_tokens: int = 24,
    clean=True,
    **kwargs,
) -> str:
    """
    augment_text - use a prompt to generate text

    :param str prompt: prompt to augment
    :param pipeline:   pipeline to use for augmentation (text-generation)
    :param int max_new_tokens: maximum number of tokens to generate, default 24
    :param bool clean: clean the generated text, default True
    :return str: augmented text
    """
    result = generator(
        prompt,
        max_new_tokens=max_new_tokens,
        **kwargs,
    )  # generate
    return (
        clean_text(result[0]["generated_text"])
        if clean
        else result[0]["generated_text"]
    )

In [5]:
import time

prompt = "I saw a dog running down the street and "  # @param {type:"string"}

kwargs = {
    "clean_up_tokenization_spaces": True,
    # your kwargs/params here (temperature, etc)
}

st = time.perf_counter()
aug_prompt = augment_text(
    prompt=prompt,
    pipeline=generator,
    **kwargs,
)
rt = round(time.perf_counter() - st, 2)

print(f"Runtime: {rt} seconds", "\n" * 2)
aug_prompt

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Runtime: 1.54 seconds 




'I saw a dog running down the street and approaching them, digital art by dan mumford and naoto hattori, fantasy art, in front of an explosion'